# Regression 

Reading: Chapter 6

This module focuses on a particular class of supervised machine learning: regression, where we are trying to discover relationships between variables in order to predict continuous quantities.  Content for this module:
* linear least-squares 
* non-linear regression models
* penalized regression models

Resources:

1. Gander, Gander and Kwok (2014), Scientific Computing: An introduction using Maple and MATLAB, Springer, https://www.springer.com/us/book/9783319043241
2. Kuhn and Johnson (2013), Applied Predictive Modeling, Springer
https://www.springer.com/us/book/9781461468486

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from scipy.linalg import solve_triangular

from sklearn import datasets, linear_model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score


## Linear Regression (Section 6.2) 

For samples with $D$ features, linear regression attempts to fit a $D$-dimensional hyperplane to the data.

In [ ]:
# Generate some data 
n = 200
x = np.random.rand(n,1) # feature 1
y = np.random.rand(n,1) # feature 2
z = x + 0.5*y + 0.1*np.random.randn(n,1) + 0.2 # response variable, z(x,y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x,y,z,c='b',marker='o')
ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')
_ = ax.set_zlabel('response')
plt.show()

Specifically, we are trying to find coefficients $a_0, a_1, a_2, \ldots, a_D$ so that the hyperplane

\begin{align}
 z(x_1,x_2,\ldots,x_D) = a_0 + a_1 x_1 + a_2 x_2 + \cdots + a_D x_D
\end{align}

models the response variable. (note, this is the generalized form of equation (6.2) in textbook).  Suppose we have $N$ samples, each with $D$ features, $\vec{x}_i = [{x_i}_1,{x_i}_2,\ldots,{x_i}_D],\, i= 1,\ldots N$. Then the hope, is that 

\begin{align}
  z({x_i}_1,{x_i}_2,\ldots,{x_i}_D) \approx y_i,\quad \text{for } i = 1,\ldots, N
\end{align}

This gives rise to an over-determined system of equations:

\begin{align}
  a_0 + a_1 x_{11} + a_2 x_{12} + \cdots + a_D x_{1D} &\approx y_1\\
  a_0 + a_1 x_{21} + a_2 x_{22} + \cdots + a_D x_{2D} &\approx y_2\\
    &\vdots \\
  a_0 + a_1 x_{N1} + a_2 x_{N2} + \cdots + a_D x_{ND} &\approx y_N\\
\end{align}

This is more easily expressed in matrix form, $X\,a = y$,

\begin{align}
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1D}\\
1 & x_{21} & x_{22} & \cdots & x_{2D}\\
1 & x_{31} & x_{32} & \cdots & x_{3D}\\
\vdots & \vdots & \vdots & \vdots & \vdots \\
1 & x_{N1} & X_{x2} & \cdots & x_{ND}
\end{bmatrix}
\begin{bmatrix}
a_0\\
a_1 \\
\vdots\\
a_D
\end{bmatrix}
\approx
\begin{bmatrix}
y_1 \\
y_2 \\
y_3 \\
\vdots \\
y_N
\end{bmatrix}
\end{align}

Here, the matrix $X$ is of size: $N \times (D+1)$, the vector of coefficients we seek is of size $(D+1)\times 1$, the right-hand side vector is of size $N \times 1$.  Here, we are imagining that $N \gg D$.  How do we solve the overdetermined system?

Well, it turns out we can't exactly most of the time, i.e., we can not find $a$ such that $X\,a =y$.  There is always some mismatch, a residual: $r = X\,a - y$.  We want to minimize this residual, for example, minimize $\|r\|^2_2 = \|X\,a - y\|^2_2$, i.e.,

$$ \min \|X\,a - y \|^2_2 = \min \left( \sum_{i = 1}^N  (X\,a - y)_i^2 \right) $$

#### Theorem 

If $X^T (X\,a - y) = 0$, then $a$ solves the linear least squares problem, i.e. $a$ minimizes $\|X\,a - y\|_2^2$

Proof: Let $c$ be any vector of size $(D+1)\times 1$.  Then $X(a+c) - y = X\,a -y + X\,c $, and

\begin{align}
\|X(a+c) - y\|_2^2 &= \left( X\,a -y + X\,c \right)^T \left(X\,a -y + X\,c \right) \\
&= (X\,a - y)^T (X\,a -y) + 2 (X\,c)^T(X\,a - y) + (X\,c)^T (X\,c) \\
&= \|X\,a - y\|_2^2 + 2c^T X^T (X\,a - y) + \|X\,c\|_2^2
\end{align}

If $X^T (X\,a - y) = 0$, then
\begin{align}
\|X(a+c) - y\|_2^2 = \|X\,a - y\|_2^2 + \|X\,c\|_2^2
\end{align}

Hence, for every $c$, we have 
\begin{align}
\|X(a+c) - y\|_2^2 \ge  \|X\,a - y\|_2^2
\end{align}

so $a$ must minimize $\|X\,a - y\|_2^2$ as required. 

The equation $X^T(X\,a - y) = 0$ is often written as

$$ X^T X\,a = X^T y, $$

called the normal equations.  There is a geometric interpretation: the vector in the range (column space of $X$) that lies closest to $y$ makes $X\,a - y$ perpendicular to the range.  Note, 
* the normal equations expresses the $N\times(D+1)$ linear least squares problem as a $(D+1)\times (D+1)$ linear system.
* the matrix $X^T X$ is symmetric
* $X^T X$ is singular . if and only if the columns of $X$ are linearly dependent, i.e., the rank of $X$ is less than $(D+1)$
* if $X^T X$ is non singular, then it is positive definite.

Observe that if $X^T X$ is non singular, then
$$ X^T X\,a = X^T y, $$
can be written as 
$$ a = (X^T X)^{-1} X^T y, $$

One sometimes defines the pseudo inverse,
$$ X^\dagger = (X^T X)^{-1} X^T.$$
Then, the least squares problem $X\,a = y$ has a solution $a = X^\dagger y$.

### QR Factorization 

Numerically, one never forms the pseudo inverse to solve the least squares problem.  Rather, we rely on a useful factorization known as the QR factorization.  Every matrix has a QR factorization, $X = QR$, where $Q$ is an orthogonal matrix, and $R$ is upper triangular.  For convenience, lets refer to
* $X$ as an $m \times n$ matrix
* $Q$ is an $m \times m$ matrix, whose columns are orthonormal to each other, i.e.:
    * $q_i \cdot q_j = 0$ if $i\neq j$
    * $q_i \cdot q_i = 1, \quad i=1,2,\ldots,m$
* $R$ is an $m \times n$ upper triangular matrix, i.e., $r_{ij} = 0$ if $i > j$.

#### Example 1
Lets switch to a toy example where we have an over-determined system, and explore the QR factorization and least-squares solution.  We will return to regression shortly. Suppose we have two packages that we wish to ship.  We measure the weight of these two packages in the office.  Package $A$ measures in at 2 pounds, package $B$ measures in at 5 pounds.  At the distribution site however, the two packages are weighed together, and the joint weight is reported at 8 pounds.  Use a least-squares solution to find a best estimate for the weight of each package.

In [ ]:
X = np.array( [ [1,0], [0,1], [1,1]])
print("our matrix X:")
print(X)
y = np.array( [[2],[5],[8]])
print("our target y:")
print(y)

In [ ]:
[Q,R] = np.linalg.qr(X, mode = 'complete')
print("Lets check that Q*R = X, up to machine precision")
print(np.matmul(Q,R))

Lets check that $Q$ has orthonormal columns

In [ ]:
print("matrix Q: ")
print(Q)
print("\nVarious inner products:")
print("q1^t q_1 = %g\n " % np.dot(Q[:,0],Q[:,0]) )
print("q1^t q_2 = %g \n" % np.dot(Q[:,0],Q[:,1]) )
print("q1^t q_3 = %g \n" % np.dot(Q[:,0],Q[:,2]) )
print("q2^t q_2 = %g \n" % np.dot(Q[:,1],Q[:,1]) )
print("q2^t q_3 = %g \n" % np.dot(Q[:,1],Q[:,2]) )
print("q3^t q_3 = %g " % np.dot(Q[:,2],Q[:,2]) )

Lets check that $R$ is upper triangular

In [ ]:
print("R = ")
print(R)

There is a reduced factorization that is more useful. $ X = \hat{Q} \hat{R}$, where $\hat{Q}$ is an $m\times n$ matrix with orthonormal columns, and $\hat{R}$ is an $n\times n$ upper triangular matrix.

In [ ]:
[Q,R] = np.linalg.qr(X, mode = 'reduced')
print("Q = ")
print(Q)
print("R = ")
print(R)
print("Lets check that Q*R = X, up to machine precision")
print(np.matmul(Q,R))

### QR Factorization + Normal Equations 

How do we use the $QR$ factorization to solve the normal equations?  Suppose we have $X = \hat{Q}\,\hat{R}$.  Observe:
\begin{align}
X^T X a &= X^T y \\
(\hat{Q}\,\hat{R})^T (\hat{Q}\,\hat{R})\, a &= (\hat{Q}\,\hat{R})^T y \\
\hat{R}^T \hat{Q}^T \hat{Q}\, \hat{R} \, a &= \hat{R}^T \hat{Q}^T y
\end{align}



If $\hat{R}^T$ exists, and observing that $\hat{Q}^T \hat{Q}$ gives an $n\times n$ identity matrix,
\begin{align}
&\implies \hat{R}^T \hat{R} \, a = \hat{R}^T \hat{Q}^T y \\
&\implies  \hat{R} \, a = \hat{Q}^T y
\end{align}

#### Example 1 (revisited)
Lets use our toy example to solve our derived normal equations, simplified using the QR factorization.

In [ ]:
# from scipy.linalg import solve_triangular

x = solve_triangular(R,np.dot(np.transpose(Q),y)) 
# default is to solve upper triangular matrix.  to solve lower triangular, add flag lower=true
print("package weights: ")
print(x)

#### Example 2 

Let's now visit regression, which is a linear least squares problem, using the diabetes dataset that is part of Scikit-learn.  

In [ ]:
# from sklearn import datasets, linear_model

# Load the diabetes dataset
diabetes = datasets.load_diabetes()
print(diabetes.DESCR)

Lets pick one feature to do a linear regression with.  (We will explore using more features shortly).  The third column is the body-mass-index, centered about the mean and scaled by the standard deviation.

In [ ]:
X = diabetes.data[:,np.newaxis,2]
y = diabetes.target[:]

plt.scatter(X,y,color='black')
plt.xlabel('scaled BMI')
plt.ylabel('diabetes progression after one year');

Lets split the data into training and test sets.

In [ ]:
# from sklearn.model_selection import train_test_split

# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
              
# from sklearn.linear_model import LinearRegression
# train regression model
regr = LinearRegression(fit_intercept=True)
regr.fit(X_train,y_train)

y_pred = regr.predict(X_test)

# plot training data, overlay with fit
plt.scatter(X_train,y_train,color='black')
plt.plot(X_test,y_pred,color='blue',linewidth=3)
plt.xlabel('scaled BMI')
_ = plt.ylabel('diabetes progression after one year')

We should of course measure the mean squared error (MSE) and the $R^2$ value.  Denote the true target as $y$, the predicted target as $\hat{y}$.  Then,

\begin{align}
  \text{MSE} = \frac{1}{n} \sum_{i = 1}^n (y_{i} - \hat{y}_{i})^2.
\end{align}

What about the $R^2$ value? If the mean can be estimated or computed,

\begin{align}
  \bar{y} = \frac{1}{n}\sum_{i=1}^n y_i,
\end{align}

then the $R^2$ value can be computed using 

\begin{align}
R^2 = 1 - \frac{\sum_{i = 1}^n (y_{i} - \hat{y}_{i})^2} {\sum_{i = 1}^n (y_{i} - \bar{y})^2}.
\end{align}

How do we interpret the $R^2$ value?
* $R^2 = 0$ indicates that the model explains none of the variability of the response data about the mean
* $R^2 = 1$ indicates that the model explains all the variability of response data about the mean

In [ ]:
# from sklearn.metrics import mean_squared_error, r2_score

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# R^2 score: 1 is perfect prediction
print('R^2 score: %.2f' % r2_score(y_test, y_pred))

The coefficients of the blue line can be obtained using

In [ ]:
print('Coefficients: %g '%  regr.coef_)
print('Intercept: %g '%  regr.intercept_)

Lets recover these coefficients using our normal equations

In [ ]:
A = np.ones( (X_train.size,2) )
A[:,np.newaxis,1]  = X_train

# find QR factorization
[Q,R] = np.linalg.qr(A, mode = 'reduced')

# solve normal equations
coeff = solve_triangular(R,np.dot(np.transpose(Q),y_train)) 
# default is to solve upper triangular matrix.  to solve lower triangular, add flag lower=true

# print coefficients
print("coefficients: ")
print(coeff)

Voila!  They match. 

#### Example 3

 We can also fit more features; hard to visualize when there are more than 2 fields.  Lets try: age and blood pressure

In [ ]:
X = diabetes.data[:,[2,3]]
y = diabetes.target[:]


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0],X[:,1],y,c='black',marker='o')
ax.set_xlabel('scaled BMI')
ax.set_ylabel('scaled blood pressure')
_ = ax.set_zlabel('diabetes progression after one year');

Let's split the data into training and test data, and fit a hyperplane to this data

In [ ]:
# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
              
# train regression model on training data
regr = LinearRegression(fit_intercept=True)
regr.fit(X_train,y_train)

y_pred = regr.predict(X_test)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_train[:,0],X_train[:,1],y_train,c='black',marker='o')
ax.plot_trisurf(X_test[:,0],X_test[:,1],y_pred) # plot hyperplane
ax.set_xlabel('scaled BMI')
ax.set_ylabel('scaled blood pressure')
_ = ax.set_zlabel('diabetes progression after one year')

print('Coefficients: ')
print(regr.coef_)
print('Intercept: %g '%  regr.intercept_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# R^2 score: 1 is perfect prediction
print('R^2 score: %.2f' % r2_score(y_test, y_pred))

Again, lets recover the hyperplane coefficients using our normal equations

In [ ]:
A = np.ones( (X_train.shape[0],3) )
A[:,np.newaxis,[1,2]]  = X_train[:,np.newaxis,[0,1]]

# find QR factorization
[Q,R] = np.linalg.qr(A, mode = 'reduced')

# solve normal equations
coeff = solve_triangular(R,np.dot(np.transpose(Q),y_train)) 
# default is to solve upper triangular matrix.  to solve lower triangular, add flag lower=true

# print coefficients 
print("coefficients: ")
print(coeff)

#### Example 4 

We can of course fit a hyperplane to all the features, though it is difficult to now visualize the 10-dimensional hyperplane. 

In [ ]:
X = diabetes.data
y = diabetes.target[:]

# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
              
# train regression model on training data
regr = LinearRegression(fit_intercept=True)
regr.fit(X_train,y_train)

y_pred = regr.predict(X_test)

print('Coefficients: ')
print(regr.coef_)
print('Intercept: %g '%  regr.intercept_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# R^2 score: 1 is perfect prediction
print('R^2 score: %.2f' % r2_score(y_test, y_pred))

and again, using our normal equations:

In [ ]:
A = np.ones( (X_train.shape[0],11) )
A[:,np.newaxis,range(1,11)]  = X_train[:,np.newaxis,range(10)]

# find QR factorization
[Q,R] = np.linalg.qr(A, mode = 'reduced')

# solve normal equations
coeff = solve_triangular(R,np.dot(np.transpose(Q),y_train)) 
# default is to solve upper triangular matrix.  to solve lower triangular, add flag lower=true

# print coefficients
print("coefficients: ")
print(coeff)

Going back to first principle allows you to fit more general functions to your data,

\begin{align}
y(\vec{x}) = a_1 f_1(\vec{x}) + a_2 f_2(\vec{x}) + \cdots + a_D f_D(\vec{x})
\end{align}

as long as we seek unknown coefficients that give a linear combination of these functions.  Examples of these more general functions include higher degree polynomials, periodic functions such as sines and cosines.

## Sparse Models (Section 6.2.3) 

If one has many features (predictors), there is a tendency to over-fit the data: one picks up noise in the system if there are features that are uninformative.    Feature selection seeks to select only the informative features while discarding non-informative ones.  The text introduces one approach known as penalization methods, where one penalizes the magnitude of coefficients of features.  Recall that in the least squares sense, we wish to minimize
$\|r\|^2_2 = \|X\,a - y\|^2_2$, i.e.,

$$ \min \|X\,a - y \|^2_2 = \min \left( \sum_{i = 1}^N  (X\,a - y)_i^2 \right) $$

Lasso Regression adds $L_1$ regularization to the minimization problem, that is, it adds a penalty equivalent to the sum of the absolute value of the coefficients

$$ \min \left(\|X\,a - y \|^2_2 + \alpha \|a\|_1\right) = \min \left(\left( \sum_{i = 1}^N  (X\,a - y)_i^2 \right) + \alpha \sum_{j=0}^{D} |a_j| \right)$$

Ridge Regression adds $L_2$ regularization to the minimization problem, that is, it adds a penalty equivalent to the sum of the squares of the coefficients

$$ \min \left(\|X\,a - y \|^2_2 + \alpha \|a\|_2^2\right) = \min \left(\left( \sum_{i = 1}^N  (X\,a - y)_i^2 \right) + \alpha \sum_{j=0}^{D} |a_j|^2 \right)$$

There are courses that cover numerical methods for solving these sort of optimization problems (MA 5630); we will not try to explain them here.  Instead, lets focus on one of these methods to better understand the role of the regularization parameter and why these penalization methods fall into the class of feature selection methods.

### Example 5

Lets use the diabetes data, all features, and apply Lasso Regression with various penalty coefficients, $\lambda$, and track how the coefficients of the hyperplane, the MSE and the $R^2$ values changes.  Note, if $\alpha = 0$ (no penalization), we recover exactly the linear regression.

In [ ]:
X = diabetes.data
y = diabetes.target[:]

# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# test the following values of lambda
alpha = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100, 1000, 10000]

# We'll store our data in a pandas data frame
import pandas as pd

#Initialize a dataframe to store the results:
col = ['MSE','R^2'] + ['a_%d'%i for i in range(11)]
ind = ['alpha = %g'%i for i in alpha]
       
results = pd.DataFrame(index=ind, columns=col)

for i in range(len(alpha)):
    # train regression model on training data
    regr = linear_model.Ridge(fit_intercept=True,alpha=alpha[i])
    regr.fit(X_train,y_train)

    y_pred = regr.predict(X_test)
    
    results.iloc[i,2] = regr.intercept_
    results.iloc[i,3:] = regr.coef_
    results.iloc[i,0] = mean_squared_error(y_test, y_pred)
    results.iloc[i,1] = r2_score(y_test, y_pred)

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
results

### Try on your own 

Exercise: 
* explore ridge regression instead of lasso regression
